In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy

In [5]:
p = numpy.array(
    [
        [11, 12, 13],
        [21, 22, 23],
        [31, 32, 33],
    ]
)


# p = numpy.array(
#     [
#         [11, 12, 13, 14, 15],
#         [21, 22, 23, 24, 25],
#         [31, 32, 33, 34, 35],
#         [41, 42, 43, 44, 45],
#         [51, 52, 53, 54, 55],
#     ]
# )

p

array([[11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]])

In [6]:
import lasp.utils
# p_shifted = lasp.utils.pad_circshift_center(p, p.shape)
center = numpy.array(p.shape) // 2
p_shifted = numpy.roll(p, -center, [0, 1])
print(p_shifted)

[[22 23 21]
 [32 33 31]
 [12 13 11]]


In [7]:
def make_bccb(p: numpy.ndarray) -> numpy.ndarray :
    
    N, M = p.shape
    
    dim = numpy.prod(numpy.array(p.shape))
    p_cpy = numpy.copy(p)
    
    p1 = numpy.reshape(p_cpy.T, newshape=dim)
    P = numpy.zeros(shape=(dim, dim))
    P[:, 0] = p1

    # print("First Boucle")
    # print(P)
    for j in range(1, M):
        for i in range(0, M*N, N):
            P[i:i+N, j] = numpy.roll(P[i:i+N, j-1], 1)
            # print(P)

    # print("Second Boucle")
    for j in range(1, M):
        P[:, N*j: N*j+N] = numpy.roll(P[:, N*(j-1): N*(j-1)+N], (N, 0), (0, 1))
        # print(P)


    return P

In [8]:
bccb = make_bccb(p_shifted)
print(bccb)


[[22. 12. 32. 21. 11. 31. 23. 13. 33.]
 [32. 22. 12. 31. 21. 11. 33. 23. 13.]
 [12. 32. 22. 11. 31. 21. 13. 33. 23.]
 [23. 13. 33. 22. 12. 32. 21. 11. 31.]
 [33. 23. 13. 32. 22. 12. 31. 21. 11.]
 [13. 33. 23. 12. 32. 22. 11. 31. 21.]
 [21. 11. 31. 23. 13. 33. 22. 12. 32.]
 [31. 21. 11. 33. 23. 13. 32. 22. 12.]
 [11. 31. 21. 13. 33. 23. 12. 32. 22.]]


In [9]:
bccb_col = bccb[:, 0]
reshaped = bccb_col.reshape(p.shape)
# print(reshaped)
print(numpy.roll(reshaped, center, [0, 1]))

[[11. 21. 31.]
 [12. 22. 32.]
 [13. 23. 33.]]


In [10]:
bccb_col = bccb.T[:, 0]
reshaped = bccb_col.reshape(p.shape)
# print(reshaped)
print(numpy.roll(reshaped, center, [0, 1]))

[[33. 23. 13.]
 [32. 22. 12.]
 [31. 21. 11.]]


In [11]:
numpy.flip(p)

array([[33, 32, 31],
       [23, 22, 21],
       [13, 12, 11]])

In [141]:
numpy.fft.fft2(p_shifted)

array([[198. +0.j        ,   0. -5.19615242j,   0. +5.19615242j],
       [  0.-51.96152423j,   0. +0.j        ,   0. +0.j        ],
       [  0.+51.96152423j,   0. +0.j        ,   0. +0.j        ]])

In [13]:
# numpy.fft.fft(bccb[:, 0])

array([198.         +0.j        ,  -2.81907786 -3.35964617j,
         2.29813333 +0.40522291j,   0.        -51.96152423j,
         0.52094453 -1.43128334j,   0.52094453 +1.43128334j,
         0.        +51.96152423j,   2.29813333 -0.40522291j,
        -2.81907786 +3.35964617j])

In [103]:
def dftmtx(n):
    h = numpy.arange(0, n)
    i, j = numpy.meshgrid(h, h)
    return numpy.exp(-2j*numpy.pi*i*j / n)

def idftmtx(n):
    h = numpy.arange(0, n)
    i, j = numpy.meshgrid(h, h)
    dft = numpy.exp(-2j*numpy.pi*i*j / n)
    return numpy.transpose(numpy.conj(dft))

# def dftmtx_fft(x):
#     n = x.shape[0]
#     return (dftmtx(n) @ x ) / numpy.sqrt(n)

# def dftmtx_ifft(x):
#     n = x.shape[0]
#     return (idftmtx @ x ) * numpy.sqrt(n)

import scipy.linalg
n = 4
sp, my = scipy.linalg.dft(n), dftmtx(n)
numpy.allclose(sp, my)

True

In [104]:
# dftmtx(2) @ idftmtx(2)

In [105]:
# idftmtx(2) @ dftmtx(2)

In [106]:
n = 5
x = numpy.random.rand(n);
print('x : {}'.format(x))

x : [0.99381834 0.95481747 0.50045651 0.92542579 0.59155773]


$$
\begin{align*}
y = Fx = fft(x) \implies x = \frac{1}{\sqrt n} F^{T} y = ifft(y) 
\end{align*}
$$

In [107]:
x_fft = numpy.fft.fft(x)
x_dft = dftmtx(n) @ x
print('FFT(x) : {}\n'.format(x_fft))
print('F @ x : {}'.format(x_dft))

FFT(x) : [3.96607585+0.j         0.31811154-0.09568986j 0.18339638-0.61768853j
 0.18339638+0.61768853j 0.31811154+0.09568986j]

F @ x : [3.96607585+0.j         0.31811154-0.09568986j 0.18339638-0.61768853j
 0.18339638+0.61768853j 0.31811154+0.09568986j]


In [110]:
x_dft_ifft = numpy.fft.ifft(x_dft)
x_dft_ifft = x_dft_ifft.real
x_dft_idft = (idftmtx(n) @ x_dft) / n
x_dft_idft = x_dft_idft.real
print('x : {}\n'.format(x))
print('IFFT(x) : {}\n'.format(x_dft_ifft))
print('F.T @ x : {}'.format(x_dft_idft))

x : [0.99381834 0.95481747 0.50045651 0.92542579 0.59155773]

IFFT(x) : [0.99381834 0.95481747 0.50045651 0.92542579 0.59155773]

F.T @ x : [0.99381834 0.95481747 0.50045651 0.92542579 0.59155773]


$$
\begin{align*}
y = F^{T}x = ifft(x) \implies x = {\sqrt n} \times F y = fft(y) 
\end{align*}
$$

In [125]:
FTx = idftmtx(n) @ x / n
FTxF = FTx @ dftmtx(n)
FTxF = FTxF.real
print('x : {}\n'.format(x))
print('F.T @ x : {}\n'.format(FTx))
print('IFFT(x) : {}\n'.format(numpy.fft.ifft(x)))
print('F.T @ x @ F: {}\n'.format(FTxF))
print('x == F.T @ x @ F : {}'.format(numpy.allclose(x, FTxF)))

x : [0.99381834 0.95481747 0.50045651 0.92542579 0.59155773]

F.T @ x : [0.79321517+0.j         0.06362231+0.01913797j 0.03667928+0.12353771j
 0.03667928-0.12353771j 0.06362231-0.01913797j]

IFFT(x) : [0.79321517+0.j         0.06362231+0.01913797j 0.03667928+0.12353771j
 0.03667928-0.12353771j 0.06362231-0.01913797j]

F.T @ x @ F: [0.99381834 0.95481747 0.50045651 0.92542579 0.59155773]

x == F.T @ x @ F : True


In [118]:
Fx = dftmtx(n) @ x
FxFT =  Fx @ idftmtx(n) / n
FxFT = FxFT.real
print('x : {}\n'.format(x))
print('F @ x : {}\n'.format(Fx))
print('F @ x @ F.T: {}\n'.format(FxFT))
print('x == F @ x @ F.T : {}'.format(numpy.allclose(x, FxFT)))

x : [0.99381834 0.95481747 0.50045651 0.92542579 0.59155773]

F @ x : [3.96607585+0.j         0.31811154-0.09568986j 0.18339638-0.61768853j
 0.18339638+0.61768853j 0.31811154+0.09568986j]

F @ x @ F.T: [0.99381834 0.95481747 0.50045651 0.92542579 0.59155773]

x == F @ x @ F.T : True


In [158]:
A = numpy.array(
    [
        [ 1, 2, 3, 4 ],
        [ 2, 1, 4, 3 ],
        [ 3, 4, 1, 2 ],
        [ 4, 3, 2, 1 ]
    ]
)
M, N = A.shape[0] / 2, A.shape[1] / 2

In [179]:
F = numpy.kron(scipy.linalg.dft(M), scipy.linalg.dft(N))
F = F
F

array([[ 1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
         1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
         1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j],
       [ 1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
         1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
         1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j],
       [ 1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j,
         1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j,
         1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j],
       [ 1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
        -0.5-8.66025404e-01j, -0.5-8.66025404e-01j, -0.5-8.66025404e-01j,
        -0.5+8.66025404e-01j, -0.5+8.66025404e-01j, -0.5+8.66025404e-01j],
       [ 1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
        -0.5-8.66025404e-01j, -0.5

In [174]:
FT = numpy.transpose(numpy.conj(F))
FT

array([[ 1.,  1.,  1.,  1.],
       [ 1., -1.,  1., -1.],
       [ 1.,  1., -1., -1.],
       [ 1., -1., -1.,  1.]])

In [175]:
D = FT @ A @ F / (M*N)
D
numpy.diag(D).real

array([10., -2., -4.,  0.])

In [176]:
# numpy.fft.fft(bccb[:, 0])

In [178]:
M, N = 3, 3
F = numpy.kron(scipy.linalg.dft(M), scipy.linalg.dft(N))
F = F
F

array([[ 1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
         1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
         1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j],
       [ 1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
         1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
         1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j],
       [ 1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j,
         1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j,
         1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j],
       [ 1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
        -0.5-8.66025404e-01j, -0.5-8.66025404e-01j, -0.5-8.66025404e-01j,
        -0.5+8.66025404e-01j, -0.5+8.66025404e-01j, -0.5+8.66025404e-01j],
       [ 1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
        -0.5-8.66025404e-01j, -0.5